## packages

In [2]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# from rich import print

In [31]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models
from ewatercycle.util import get_time

In [6]:
from shutil import copytree
from unittest import mock

import pytest
import xarray as xr

In [9]:
path = Path.cwd()
forcing_path = path / "Forcing"

In [10]:
camels_path = forcing_path  / 'Camels'

In [14]:
datasets = list(camels_path.glob("camels_03439000*.nc"))

### Funtions to test

In [7]:
from ewatercycle._forcings.caravan import (
    CaravanForcing,
    get_shapefiles,
    extract_basin_shapefile,
    crop_ds,
)

In [39]:
@pytest.fixture
def mock_retrieve():
    with mock.patch("xarray.Dataset") as mock_class:
        mock_class.return_value = xr.open_dataset(forcing_path / "test_caravan_file.nc")
        yield mock_class

In [16]:
xr.open_dataset(datasets[0]) 

<xarray.Dataset> Size: 36kB
Dimensions:         (time: 2954)
Coordinates: (12/18)
  * time            (time) datetime64[ns] 24kB 1997-08-01 ... 2005-09-01
    basin_id        |S64 64B ...
    timezone        |S64 64B ...
    name            |S64 64B ...
    country         |S64 64B ...
    lat             float64 8B ...
    ...              ...
    moisture_index  float64 8B ...
    seasonality     float64 8B ...
    high_prec_freq  float64 8B ...
    high_prec_dur   float64 8B ...
    low_prec_freq   float64 8B ...
    low_prec_dur    float64 8B ...
Data variables:
    tasmax          (time) float32 12kB ...

In [17]:
COMMON_URL = "ca13056c-c347-4a27-b320-930c2a4dd207"
OPENDAP_URL = f"https://opendap.4tu.nl/thredds/dodsC/data2/djht/{COMMON_URL}/1/"
SHAPEFILE_URL = (
    f"https://data.4tu.nl/file/{COMMON_URL}/bbe94526-cf1a-4b96-8155-244f20094719"
)
basin_id = "camels_03439000"
dataset = basin_id.split("_")[0]

In [41]:
run = False
if run:
    ds = xr.open_dataset(f"{OPENDAP_URL}{dataset}.nc")
    start_time = '1981-01-01T00:00:00.000000000Z'
    end_time = '1982-01-01T00:00:00.000000000Z'
    ds_basin = ds.sel(basin_id=["camels_01022500".encode(),"camels_03439000".encode()])
    ds_basin_time = crop_ds(ds_basin, start_time, end_time)
    ds_basin_time.to_netcdf(forcing_path / "test_caravan_file.nc")
else: 
    ds_basin_time = xr.open_dataset(forcing_path / "test_caravan_file.nc")

In [26]:
def crop_ds(ds: xr.Dataset, start_time: str, end_time: str) -> xr.Dataset:
    """Crops dataset based on time."""
    get_time(start_time), get_time(end_time)  # if utc, remove Z to parse to np.dt64
    start, end = np.datetime64(start_time[:-1]), np.datetime64(end_time[:-1])
    return ds.isel(
        time=(ds["time"].to_numpy() >= start) & (ds["time"].to_numpy() <= end)
    )


In [42]:
ds_basin_time

<xarray.Dataset> Size: 118kB
Dimensions:                             (time: 366, basin_id: 2)
Coordinates:
  * time                                (time) datetime64[ns] 3kB 1981-01-01 ...
  * basin_id                            (basin_id) |S64 128B b'camels_0102250...
Data variables: (12/55)
    timezone                            (basin_id) |S64 128B ...
    name                                (basin_id) |S64 128B ...
    country                             (basin_id) |S64 128B ...
    lat                                 (basin_id) float64 16B ...
    lon                                 (basin_id) float64 16B ...
    area                                (basin_id) float64 16B ...
    ...                                  ...
    volumetric_soil_water_layer_2_max   (basin_id, time) float32 3kB ...
    volumetric_soil_water_layer_3_max   (basin_id, time) float32 3kB ...
    volumetric_soil_water_layer_4_max   (basin_id, time) float32 3kB ...
    total_precipitation_sum             (basin_id, time) float32 3kB ...
    potential_evaporation_sum           (basin_id, time) float32 3kB ...
    streamflow                          (basin_id, time) float32 3kB ...
Attributes:
    history:  Wed Mar 27 16:11:00 2024: /usr/bin/ncap2 -s time=double(time) -...
    NCO:      netCDF Operators version 5.0.6 (Homepage = http://nco.sf.net, C...